In [7]:
import requests      
import numpy as np   
import pandas as pd  
import time          

from bs4 import BeautifulSoup
import re

In [8]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36'

In [9]:
# !/usr/local/anaconda3/bin/pip install fake_useragent

In [10]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [11]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 185.220.101.70


In [14]:
MAIN_LINK = "http://k.intimcity.nl/persons.php?index={}"

In [15]:
page_link = "http://k.intimcity.nl/persons.php?id=352916"

PERSON_LINK = "http://k.intimcity.nl/persons.php?id={}"

In [9]:
main_link.format(1)

'http://k.intimcity.nl/persons.php?index=1'

In [16]:
def get_soup(link):
    headers = {'User-Agent': UserAgent().chrome}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [17]:
def get_main_page(page_number, main_link=MAIN_LINK):
    main_page = get_soup(main_link.format(page_number))
    return main_page

In [18]:
def get_max_main_page(main_page):
    table = main_page.find('table', attrs={'style':'margin: auto; border-spacing: 3px;'})
    max_page = int(table.findAll('a', attrs={'class':'index'})[-2].text)
    return max_page

In [21]:
def get_persons_links(current_main_page, person_link=PERSON_LINK):
    persons_links = []
    page_ids = current_main_page.findAll('a', attrs={'class':'x'})
    for page_id in page_ids:
        page_id = re.findall(r'\d+', page_id.get('onclick'))[0]
        link = person_link.format(page_id)
        persons_links.append(link)
    return persons_links

In [22]:
current_main_page = get_main_page(1)
current_persons_links = get_persons_links(current_main_page)

In [23]:
len(current_persons_links)

50

In [24]:
len(set(current_persons_links))

50

In [355]:
def get_name(current_person_page):
    return {'name':current_person_page.find('h2').text}

In [424]:
def get_text_description(current_person_page):
    """Description written by seller"""
    description = current_person_page.find('td', attrs={'class':'ar13'})
    description = description.text if description else None
    return {'text_description': description}

In [331]:
def get_parameter_table(current_person_page):
    """All parameters (weight, height, address, etc.)"""
    parameter_table = current_person_page.find("table", attrs={'style':'width: 100%; border-spacing: 1px;'})
    parameter_table = pd.read_html(str(parameter_table))[0]
    parameter_table = parameter_table.set_index(0)
    parameter_table = parameter_table.to_dict()[1]
    return parameter_table

In [152]:
def get_prices_dict(current_person_page, flatten=True):
    """All prices (per hour, day/night, etc.)"""
    def _flatten_prices_dict(prices_dict):
        flattened_prices = {}
        for k, v in prices_dict.items():
            if isinstance(v, dict):
                for nested_k, nested_v in v.items():
                    if isinstance(nested_v, dict):
                        for nested_2_k, nested_2_v in nested_v.items():
                            flattened_prices["_".join([k, nested_k, nested_2_k])] = nested_2_v
                    else:
                        flattened_prices["_".join([k, nested_k])] = nested_v
            else:
                flattened_prices[k] = v
        return flattened_prices
    
    def _clean_number(df, row_index, col_index):
        try:
            string = df.iloc[row_index, col_index]
        except:
            return None
        try:
            return float("".join(re.findall(r'\d+', string)))
        except:
            return string
           
    prices = current_person_page.find(
        'table', 
        attrs={
            'style':'width: 440px; margin: 10px auto; background-color: #CCC; text-align: center; border-spacing: 1px;'
        }
    )
    prices = pd.read_html(str(prices))[0]
        
    prices_dict = {
        "апартаменты":{
            "днём": {
                "час": _clean_number(prices, 2, 1),
                "2часа": _clean_number(prices, 2, 2)
            },
            "ночью": {
                "час": _clean_number(prices, 2, 3),
                "2часа": _clean_number(prices, 2, 4)
            }
        },
        "выезд":{
            "днём": {
                "час": _clean_number(prices, 3, 1),
                "2часа": _clean_number(prices, 3, 2)
            },
            "ночью": {
                "час": _clean_number(prices, 3, 3),
                "2часа": _clean_number(prices, 3, 4)
            }
        },
        "контактов_в_час": _clean_number(prices, 4, 1),
        "анал": _clean_number(prices, 4, 4),
        "мбр": _clean_number(prices, 5, 4),
        "comments": _clean_number(prices, 6, 0)
    }
    
    if flatten:
        prices_dict = _flatten_prices_dict(prices_dict)
        
    return prices_dict

In [420]:
def get_services_dict(current_person_page):
    """Full list with services"""
    
    services = current_person_page.find("table", attrs={'style':'width: 100%; border-spacing: 5px;'})
    # boolean mask for cheked boxes
    services_bool = services.findAll('input', attrs={'type':'checkbox'})[:49]
    services_bool = [True if 'checked' in str(x) else False for x in services_bool]

    services_names = services.findAll('a')
    services_names = np.array([name.text for name in services_names])

    services_names = list(services_names[services_bool])
    return {'services':services_names, 'number_of_services':len(services_names)}

In [398]:
def get_photo_links(current_person_page):
    """Links to photos"""
    photo_links = current_person_page.find("div", attrs={'class':'highslide-gallery'})
    photo_links = photo_links.findAll('a')
    photo_links = [link.get("href") for link in photo_links]
    return {'photo_links':photo_links}

In [415]:
def get_checks_reviews(current_person_page):
    """Comments and reviews"""
    comments = current_person_page.find('div', attrs={'class':'noprint'}) 
    
    number_of_checks = comments.find('table', attrs={'style':'margin: auto;'})
    number_of_checks = len(number_of_checks.findAll('div', attrs={'class':'rnd5'})) if number_of_checks else 0
    # reviews from users
    reviews = comments.find('table', attrs={'style':'width: 440px; margin: 10px auto 0 auto;'})    
    reviews = reviews.findAll('div', attrs={'class':'rnd5'}) if reviews else []

    clean_reviews = []
    for review in reviews:
        span_tag = review.find('span')
        review_text = span_tag.next_sibling if span_tag else None
        try:
            clean_text = review_text.text
        except:
            clean_text = review_text
        if clean_text:
            clean_reviews.append(clean_text)
        
    number_of_reviews = len(clean_reviews)
    
    result = {
        'number_of_checks':number_of_checks,
        'number_of_reviews':number_of_reviews,
        'reviews_text':clean_reviews
    }
    return result

In [425]:
full_person_data = {}

name = get_name(current_person_page)
parameter_table = get_parameter_table(current_person_page)
text_description = get_text_description(current_person_page)
prices = get_prices_dict(current_person_page)
services = get_services_dict(current_person_page)
photo_links = get_photo_links(current_person_page)
checks_reviews = get_checks_reviews(current_person_page)

In [403]:
def get_full_person_data(current_person_page):
    full_person_data = {}
    full_person_data.update(get_name(current_person_page))
    full_person_data.update(get_parameter_table(current_person_page))
    full_person_data.update(get_text_description(current_person_page))
    full_person_data.update(get_prices_dict(current_person_page))
    full_person_data.update(get_services_dict(current_person_page))
    full_person_data.update(get_photo_links(current_person_page))
    full_person_data.update(get_checks_reviews(current_person_page))
    return full_person_data

In [431]:
from tqdm import tqdm_notebook
import time

In [430]:
person_link

'http://k.intimcity.nl/persons.php?id=353687'

In [432]:
full_data = {}

for i, person_link in tqdm_notebook(enumerate(current_persons_links)):
    current_person_page = get_soup(person_link)
    full_person_data = get_full_person_data(current_person_page)
    full_data[i] = full_person_data
    checkIP()
    time.sleep(1)

Current IP Address: 91.219.237.244
Current IP Address: 91.219.237.244
Current IP Address: 91.219.237.244
Current IP Address: 91.219.237.244
Current IP Address: 162.247.74.200
Current IP Address: 162.247.74.200
Current IP Address: 162.247.74.200
Current IP Address: 162.247.74.200
Current IP Address: 51.15.167.49
Current IP Address: 51.15.167.49
Current IP Address: 51.15.167.49
Current IP Address: 185.220.101.1
Current IP Address: 185.220.101.1
Current IP Address: 185.220.101.1
Current IP Address: 185.220.101.1
Current IP Address: 185.220.101.1
Current IP Address: 192.160.102.168
Current IP Address: 192.160.102.168
Current IP Address: 192.160.102.168
Current IP Address: 162.247.74.213
Current IP Address: 109.72.83.220
Current IP Address: 109.72.83.220
Current IP Address: 109.72.83.220
Current IP Address: 109.72.83.220
Current IP Address: 141.255.162.34
Current IP Address: 141.255.162.34
Current IP Address: 141.255.162.34
Current IP Address: 141.255.162.34
Current IP Address: 171.25.193.2

In [439]:
pd.DataFrame(full_data).T.head(2)

,comments,name,number_of_checks,number_of_reviews,number_of_services,photo_links,reviews_text,services,text_description,Вес,...,выезд_ночью_2часа,выезд_ночью_час,контактов_в_час,мбр,nan,Адрес,Рейтинг,Не старше,E-mail,Девушка месяца - АПРЕЛЬ 2016
0,Дополнительные услуги обсуждаю при личной встр...,Дарья,1,5,12,[https://intimka.nl/Persons_big/374085/5291699...,"[Шикарная девушка., 😘, Девочка виденье! Просто...","[Секс классический, Минет в презервативе, Мине...",ИНДИВИДУАЛЬНО!Реальная - в точности как на фот...,60,...,Выезда нет,Выезда нет,2,2000,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Самая натуральная!,33,43,40,[https://intimka.nl/Persons_big/284133/8951558...,"[Уверен её там нет, Восхитительная женщина. Хо...","[Секс классический, Секс анальный, Секс группо...","Я Богиня секса, девушка по имени Луна!\r\nЗага...",56,...,30000,-,"*Выезд только в квартиры, гостиницы",2000,NaN,NaN,NaN,NaN,NaN,NaN
